# ☁️ 08. 워드 클라우드 (Word Cloud): 글자로 그림 그리기

## 1. 개요
수천 개의 리뷰나 뉴스 기사를 다 읽기 귀찮을 때, **"가장 많이 나온 단어가 뭐지?"** 하고 한눈에 보고 싶으신가요?
많이 언급된 단어일수록 데이터를 **크고 볼드체로** 보여주는 시각화 방법인 **워드 클라우드**를 배워봅시다.

> **활용 예시**:
> - 대통령 연설문 분석 (가장 많이 쓴 단어는 '국민'? '경제'?)
> - 상품 리뷰 분석 (가장 많이 나온 말은 '좋아요'? '배송이 느려요'?)

---

## 2. 작업 순서 (레시피)

1. **재료 준비**: 텍스트 데이터 (예: 스팸 문자 내역)
2. **손질하기 (토큰화)**: 문장을 단어로 자릅니다. 한국어는 **명사(Noun)**만 뽑는 게 제일 깔끔합니다.
3. **불순물 제거 (불용어 처리)**: '그', '저', '것' 같은 무의미한 단어를 뺍니다.
4. **계량하기 (빈도수 계산)**: 단어 개수를 셉니다.
5. **요리하기 (시각화)**: 그림으로 그립니다.

In [ ]:
# 실습을 위해 공개된 '스팸 메일 데이터셋'을 다운로드합니다.
# gdown 라이브러리가 없다면: !pip install gdown wordcloud
import gdown 
import os
import pandas as pd

file_id = '1yXNdduc6iAxtZo0BMuGDttVW7A90mFQ-'
url = f'https://drive.google.com/uc?id={file_id}'
output = 'sms_spam.csv'

if not os.path.exists(output):
    print("데이터 다운로드 중...")
    gdown.download(url, output, quiet=False)
else:
    print("데이터가 이미 있습니다.")

# 데이터 열어보기
spam_df = pd.read_csv('sms_spam.csv')
print("데이터 개수:", len(spam_df))
spam_df.head()  # 상위 5개 출력

## 3. 핵심 단어 뽑기 (명사 추출)

한국어에서 내용은 대부분 **명사**에 들어있습니다.
"나는 맛있는 사과를 먹었다" ➡️ `나`, `사과` (핵심)

In [ ]:
from konlpy.tag import Okt

# 예시로 첫 번째 문자 내용만 분석해봅시다.
sample_text = spam_df['CN'][0]
print("분석할 문자:", sample_text)

okt = Okt()
nouns = okt.nouns(sample_text)
print("명사 추출 결과:", nouns)

## 4. 빈도수 세기

모든 문자 데이터를 합쳐서 어떤 단어가 많이 나왔는지 세어봅시다.

In [ ]:
from collections import Counter

# 1. 모든 문자 내용을 하나의 거대한 텍스트로 합치거나 리스트로 모음
all_nouns = []
for text in spam_df['CN']:
    # 각 문자마다 명사 추출해서 리스트에 추가
    # (시간이 좀 걸릴 수 있습니다)
    all_nouns.extend(okt.nouns(text))

# 2. 개수 세기
count = Counter(all_nouns)
print("가장 많이 나온 단어 Top 10:", count.most_common(10))

## 5. 불필요한 단어 빼기 (불용어 제거)
Top 10을 보니 '원', '번', '수' 같은 별 의미 없는 단어가 많네요. 빼줍시다.

In [ ]:
# 제외할 단어 리스트
stop_words = ['원', '번', '수', '것', '및', '더', '를', '을']

# 1글자 단어도 보통 의미가 없으니 같이 빼줍니다.
final_nouns = []
for noun in all_nouns:
    if noun not in stop_words and len(noun) > 1:
        final_nouns.append(noun)

final_count = Counter(final_nouns)
print("정제 후 Top 10:", final_count.most_common(10))

## 6. 그림 그리기 (WordCloud)

⚠️ **주의**: 한글 폰트를 설정해주지 않으면 글자가 `□□□` 네모로 깨져서 나옵니다!

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import platform

# 1. 폰트 경로 찾기 (운영체제별)
if platform.system() == 'Darwin': # 맥
    font_path = '/System/Library/Fonts/AppleSDGothicNeo.ttc'
elif platform.system() == 'Windows': # 윈도우
    font_path = 'C:/Windows/Fonts/malgun.ttf'
else: # 리눅스/코랩
    font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'

# 2. 워드클라우드 만들기
# background_color: 배경색 (검정/하양 추천)
wc = WordCloud(
    font_path=font_path,
    width=800,
    height=800,
    background_color='white'
)

# 빈도수 딕셔너리를 넣어줍니다.
wc.generate_from_frequencies(final_count)

# 3. 화면에 띄우기
plt.figure(figsize=(10, 10))
plt.imshow(wc)
plt.axis('off') # x축, y축 눈금 끄기
plt.show()

## 7. 꿀팁: 예쁜 모양으로 만들기 (마스킹)

하트 모양, 별 모양 이미지를 준비하면 그 모양대로 글자를 채울 수 있습니다.
(`mask` 옵션 사용)

In [ ]:
# 코드는 주석 처리해둡니다. (이미지가 필요해서)

# import numpy as np
# from PIL import Image
# mask_img = np.array(Image.open('heart.png'))
# wc = WordCloud(..., mask=mask_img)
